In [ ]:
!pip install BeautifulSoup4
!pip install lxml
!pip install pandas
!pip install requests

In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs
import csv
import datetime
import pandas as pd
import re

In [ ]:
import re

def format_tanggal(range):
    test_date = datetime.datetime.strptime(f"16-{range}-2024", "%d-%m-%Y")
    K = 7
    date_generated = pd.date_range(test_date, periods=K)
    return date_generated

def sc_tempo():
    with open('dataset_sum_tempo1.csv', 'a') as file:
        wr = csv.writer(file, delimiter=',')
        wr.writerow(['title', 'released', 'url', 'summary', 'content'])
        count = 0
        for i in range(7, 8):
            date_generate = format_tanggal(i)
            formattgl = [tgl.strftime("%Y-%m-%d") for tgl in date_generate]
            for tgl in formattgl:
                url_ = f'https://www.tempo.co/indeks/{tgl}/'
                ge_ = req.get(url_)
                if ge_.status_code == 200:
                    sop_ = bs(ge_.text, 'lxml')
                else:
                    print(f"Failed to retrieve {url_}, status code: {ge_.status_code}")
                    continue

                contai = sop_.find_all('main', class_='main-left')
                if not contai:
                    print(f"No 'main-left' elements found on {url_}")
                    continue

                for x in contai:
                    text_card = x.find_all('article', class_="text-card")
                    if not text_card:
                        print(f"No 'text-card' elements found on {url_}")
                        continue

                    for z in text_card:
                        try:
                            title = z.find('h2', class_='title').find('a').text
                            link = z.find('h2', class_='title').find('a')['href']

                            # Extract summary
                            summary = z.find('p').text if z.find('p') else 'No summary available'

                            ge_a = req.get(link).text
                            sop_a = bs(ge_a, 'lxml')

                            # Mencari elemen tanggal
                            time_element = sop_a.find('p', class_='date margin-bottom-sm')
                            if time_element:
                                time_text = time_element.text.strip()

                                time_parts = time_text.split()
                                hari = time_parts[1]
                                bulan = {
                                    'Januari': 1, 'Februari': 2, 'Maret': 3, 'April': 4,
                                    'Mei': 5, 'Juni': 6, 'Juli': 7, 'Agustus': 8,
                                    'September': 9, 'Oktober': 10, 'November': 11, 'Desember': 12
                                }
                                bulan_str = time_parts[2]
                                bulan_int = bulan.get(bulan_str, 0)
                                tahun = time_parts[3]
                                jam = time_parts[4]

                                # Format tanggal
                                formatted_date = f"{hari}/{bulan_int}/{tahun} {jam}"

                                # Ambil konten artikel
                                badan = sop_a.find('div', id='isi')

                                # Hapus semua elemen "Baca Juga"
                                baca_juga_list = badan.find_all('div', class_='bacajuga')
                                for baca_juga in baca_juga_list:
                                    baca_juga.decompose()

                                isi = [p.text for p in badan.find_all('p')]
                                ress_isi = ' '.join(isi)

                                # Menggunakan regex untuk memotong "TEMPO.CO, Nama Kota
                                pattern = r"TEMPO.CO, [A-Za-z\s]+ -\s*"
                                ress_isi = re.sub(pattern, '', ress_isi, 1)

                                # Memotong konten sebelum "Pilihan Editor:" atau "Pilihan editor:"
                                pilihan_editor_pattern = r"[Pp]ilihan [Ee]ditor:"
                                pilihan_editor_idx = re.search(pilihan_editor_pattern, ress_isi)
                                if pilihan_editor_idx:
                                    ress_isi = ress_isi[:pilihan_editor_idx.start()]

                                # Hapus kalimat tentang kontributor penulisan artikel
                                kontributor_pattern = r"[Nn]ama orang berkontribusi|[Kk]ontributor.*"
                                ress_isi = re.sub(kontributor_pattern, '', ress_isi)

                                # Hapus blok nama kontributor
                                nama_kontributor_pattern = r"([A-Z\s]+\|)+\s*[A-Z\s]+"
                                ress_isi = re.sub(nama_kontributor_pattern, '', ress_isi)

                                wr.writerow([title, formatted_date, link, summary, ress_isi])  # Added 'summary' to CSV
                                count += 1
                                print(f'[{count}][{formatted_date}][TEMPO] {title}')
                            else:
                                print(f"No date found on {link}")
                        except Exception as e:
                            print(f"Error parsing article: {e}")

sc_tempo()


[1][21/9/2024 23:14][TEMPO] Breaking News: Pilot Susi Air Tiba di Jakarta Tumpangi Pesawat TNI AU
[2][21/9/2024 23:00][TEMPO] Struktur Tim Pemenangan Ridwan Kamil-Suswono Diumumkan Usai KPU Tetapkan Nomor Urut
[3][21/9/2024 22:49][TEMPO] Pramono Anung Bicara Strategi Kampanyenya dalam Pilgub Jakarta 2024
[4][21/9/2024 22:46][TEMPO] Tak Bisa Sembuh, Ini yang Perlu Dilakukan Penderita Gangguan Pencernaan IBS
[5][21/9/2024 22:40][TEMPO] Sebelum Dibebaskan, Pilot Susi Air Sempat Minta Dua Hal Ini
[6][21/9/2024 22:36][TEMPO] Ini Kronologi Penyanderaan sampai Pembebasan Pilot Susi Air  Philip Mark Mehrtens
[7][21/9/2024 22:31][TEMPO] Sediakan 1.334 Kendaraan, Dishub Sumut Klaim Sukses Melayani Transportasi PON XXI
[8][21/9/2024 22:25][TEMPO] Relawan Berharap Ridwan Kamil-Suswono Perpanjang Kontrak Tenaga PPSU Jadi 5 Tahun
[9][21/9/2024 22:06][TEMPO] Hasil Sprint Race MotoGP Emilia Romagna 2024 dan Klasemennya: Francesco Bagnaia Juara, Jorge Martin Finis Kedua, Marc Marquez Keempat
[10][21/9/